## Pump it up! Modeling work

In [1]:
# use pandas to read in data
import pandas as pd

df_X = pd.read_csv('../data/train_X.csv')
df_y = pd.read_csv('../data/train_y.csv')
df = pd.merge(df_y, df_X, on='id')

# -- convert functional to 2, non-functional to 0, functional-needs-repaird to 1

df.loc[:, 'status'] = df.status_group
df.loc[df.status.str.startswith('functional needs'), 'status'] = 1
df.loc[df.status_group.str.startswith('non'), 'status'] = 0
df.loc[~df.status.str.startswith('functional').isnull(), 'status'] = 2
df.status = df.status.astype(int)

# -- show df.corr()
df.corr()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,status
id,1.000000,-0.005321,-0.004692,-0.001348,0.001718,-0.002629,-0.003028,-0.003044,-0.002813,-0.002082,-0.004049
amount_tsh,-0.005321,1.000000,0.076650,0.022134,-0.052670,0.002944,-0.026813,-0.023599,0.016288,0.067915,0.053702
gps_height,-0.004692,0.076650,1.000000,0.149155,-0.035751,0.007237,-0.183521,-0.171233,0.135003,0.658727,0.114029
longitude,-0.001348,0.022134,0.149155,1.000000,-0.425802,0.023873,0.034197,0.151398,0.086590,0.396732,-0.004366
latitude,0.001718,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.221018,-0.201020,-0.022152,-0.245278,0.014547
num_private,-0.002629,0.002944,0.007237,0.023873,0.006837,1.000000,-0.020377,-0.004478,0.003818,0.026056,0.005021
region_code,-0.003028,-0.026813,-0.183521,0.034197,-0.221018,-0.020377,1.000000,0.678602,0.094088,0.031724,-0.108640
district_code,-0.003044,-0.023599,-0.171233,0.151398,-0.201020,-0.004478,0.678602,1.000000,0.061831,0.048315,-0.065687
population,-0.002813,0.016288,0.135003,0.086590,-0.022152,0.003818,0.094088,0.061831,1.000000,0.260910,0.017759
construction_year,-0.002082,0.067915,0.658727,0.396732,-0.245278,0.026056,0.031724,0.048315,0.260910,1.000000,0.043342


the highest abs|correlation| is gps_height for the continuous variable
it does not say much

Initial exploration on the data

In [5]:
# import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
import patsy

I am having my first trial by just removing features that have too many
dummies

In [7]:
df.head()

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status
0,69572,functional,6000,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2
1,8776,functional,0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2
2,34310,functional,25,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,2
3,67743,non functional,0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,0
4,19728,functional,0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2


In [11]:
cols = list(df.columns.values)

# keep columns
cols_keep = []
for c in df:
    if df[c].dtype in [int, float]:
        cols_keep.append(c)
    elif df[c].dtype == object:
        if df[c].nunique() < 20:
            cols_keep.append(c)

# remove the labels
for to_remove in ['id', 'status', 'status_group']:
    cols_keep.remove(to_remove)

# convert df to X, y by patsy
r_formula = 'status ~' + ' + '.join(cols_keep)
y, X = patsy.dmatrices(r_formula, df)

In [12]:
y

DesignMatrix with shape (49841, 1)
  status
       2
       2
       0
       2
       0
       0
       2
       2
       2
       2
       2
       2
       0
       0
       2
       2
       1
       2
       2
       1
       2
       0
       2
       2
       2
       1
       2
       2
       0
       2
  [49811 rows omitted]
  Terms:
    'status' (column 0)
  (to view full data, use np.asarray(this_obj))

In [13]:
X

DesignMatrix with shape (49841, 141)
  Columns:
    ['Intercept',
     'basin[T.Lake Nyasa]',
     'basin[T.Lake Rukwa]',
     'basin[T.Lake Tanganyika]',
     'basin[T.Lake Victoria]',
     'basin[T.Pangani]',
     'basin[T.Rufiji]',
     'basin[T.Ruvuma / Southern Coast]',
     'basin[T.Wami / Ruvu]',
     'public_meeting[T.True]',
     'scheme_management[T.None]',
     'scheme_management[T.Other]',
     'scheme_management[T.Parastatal]',
     'scheme_management[T.Private operator]',
     'scheme_management[T.SWC]',
     'scheme_management[T.Trust]',
     'scheme_management[T.VWC]',
     'scheme_management[T.WUA]',
     'scheme_management[T.WUG]',
     'scheme_management[T.Water Board]',
     'scheme_management[T.Water authority]',
     'permit[T.True]',
     'extraction_type[T.cemo]',
     'extraction_type[T.climax]',
     'extraction_type[T.gravity]',
     'extraction_type[T.india mark ii]',
     'extraction_type[T.india mark iii]',
     'extraction_type[T.ksb]',
     'extraction_t

In [16]:
def split_n_fit(model, X, y):
    """ given model, X, y, print score of the fit on test """
    X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), random_state=42)
    model.fit(X_train, y_train)
    print
    print '{}'.format(model).split('(')[0]
    print model.score(X_test, y_test)


for model in [LogisticRegression(), DecisionTreeClassifier(),
              KNeighborsClassifier(), GaussianNB(), RandomForestClassifier()]:
    split_n_fit(model, X, y)


LogisticRegression
0.710456624669

DecisionTreeClassifier
0.756038841184

KNeighborsClassifier
0.690875531659

GaussianNB
0.49394109622

RandomForestClassifier
0.78677473718


In [17]:
# it looks RandomForest does us better, let's try a new parameter

model = RandomForestClassifier(n_estimators=200)
split_n_fit(model, X, y)


RandomForestClassifier
0.805633576759


In [18]:
# it looks RandomForest does us better, let's try a new parameter

model = RandomForestClassifier(n_estimators=300)
split_n_fit(model, X, y)


RandomForestClassifier
0.804590321804


In [26]:
from sklearn.cross_validation import KFold

model = RandomForestClassifier(n_estimators=200)
y = y.ravel()
for train_index, test_index in KFold(len(y), n_folds=5):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    model.fit(X_train, y_train)
    print model.score(X_test, y_test)
    

0.802086468051
0.79695024077
0.802668539326
0.803370786517
0.799257624398


In [25]:
KFold?